## 경기데이터드림 OpenApi 응답 명세
- [경기데이터드림 OpenApi 응답 명세 URL](https://data.gg.go.kr/portal/data/service/selectServicePage.do?page=1&rows=10&sortColumn=&sortDirection=&infId=V3FXHJNFCLAVBMQTBRRI29844183&infSeq=1&order=&cateId=GG29)
- csv 파일 다운로드 > csv 파일 utf8로 로컬저장 > db 스키마 생성 > db 저장 > db 분석

In [1]:
import pandas as pd
import numpy as np
from util.myUtil import MySQLDao
dao = MySQLDao()

### [문제-01] 데이터 구조(스펙)을 데이터프레임 형태로 변환하시오. 단, 데이터 출력은 좌측정렬 하시오.
- [데이터 구조확인](https://data.gg.go.kr/portal/data/service/selectServicePage.do?page=1&rows=10&sortColumn=&sortDirection=&infId=V3FXHJNFCLAVBMQTBRRI29844183&infSeq=2&order=&loc=&cateId=GG29)

In [19]:
spec = pd.DataFrame([{"영문명":"INF_ID","한글명":"공공데이터ID"},{"영문명":"INF_NM","한글명":"공공데이터명"},{"영문명":"CALL_URL","한글명":"호출URL"},{"영문명":"COL_ID","한글명":"항목ID"},{"영문명":"COL_NM","한글명":"항목명"},{"영문명":"COL_TYPE","한글명":"항목타입"},
{"영문명":"REQ_YN","한글명":"요청인자여부"}])
spec.style.set_properties(**{'text-align': 'left'}) # 데이터 좌측정렬

,영문명,한글명
0,INF_ID,공공데이터ID
1,INF_NM,공공데이터명
2,CALL_URL,호출URL
3,COL_ID,항목ID
4,COL_NM,항목명
5,COL_TYPE,항목타입
6,REQ_YN,요청인자여부


### [문제-02] csv 파일을 받은 받아서, 인코딩을 utf-8로 변경하여 저장하시오.(cp949 > utf-8)
- [다운로드 페이지](https://data.gg.go.kr/portal/data/service/selectServicePage.do?page=1&rows=10&sortColumn=&sortDirection=&infId=V3FXHJNFCLAVBMQTBRRI29844183&infSeq=1&order=&loc=&cateId=GG29) CSV 파일 다운로드

In [ ]:
# df = pd.read_csv("data/02_경기데이터드림OpenApi응답명세.csv", encoding="cp949")
# df.to_csv("data/02_경기데이터드림OpenApi응답명세.csv", encoding="utf-8", index=False) # 데이터프레임에서 자동 생성한 index 제거

### [문제-03] 저장한 csv 파일을 데이터프렘으로 변환해서 출력하시오.

In [2]:
df = pd.read_csv("data/02_경기데이터드림OpenApi응답명세.csv")
df.head(2)

,공공데이터ID,공공데이터명,호출URL,항목ID,항목명,항목타입,요청인자여부
0,KE22KP2IJFD15YD0W4BU29615963,0세아 전용 어린이집 현황,https://openapi.gg.go.kr/Age0PrvuuseKidgartn,SIGUN_CD,시군코드,TEXT,N
1,KE22KP2IJFD15YD0W4BU29615963,0세아 전용 어린이집 현황,https://openapi.gg.go.kr/Age0PrvuuseKidgartn,SIGUN_NM,시군명,TEXT,Y


### [문제-04] 각 열의 문자열 최대 길이를 구하시오.

In [17]:
# 문자열 길이 max 구하기
items = df.columns.values
def get_text_max(name):
    len_max = 0
    df1 = df.filter([name])
    for v in df1.values:
        if len(v[0]) > len_max:
            len_max = len(v[0])
    return len_max
for v in items:
    print(v,' max: ', get_text_max(v))

공공데이터ID  max:  28
공공데이터명  max:  49
호출URL  max:  55
항목ID  max:  30
항목명  max:  30
항목타입  max:  6
요청인자여부  max:  1


### [문제-05] 테이블을 생성하시오. API_MST(마스터), API_DTL(상세), API_TMP(임시)

In [ ]:
# 1.1. API_MST 삭제
dao.execute("drop table API_MST")
# 1.2. API_MST 생성
dao.execute("""
CREATE TABLE API_MST (
  API_MST_ID INT unsigned not null auto_increment COMMENT 'API_마스터_기본키',
  INF_ID CHAR(28) NOT NULL                        COMMENT '공공데이터ID',
  INF_NM VARCHAR(49) not null                     COMMENT '공공데이터명',
  CALL_URL VARCHAR(55) not null                   COMMENT '호출URL',
  PRIMARY KEY (API_MST_ID)
)
COMMENT='API_마스터'
""")

# 2.1. API_DTL 삭제
dao.execute("drop table API_DTL")
# 2.2. API_DTL 생성
dao.execute("""
CREATE TABLE API_DTL (
  API_MST_ID INT unsigned not null                COMMENT 'API_마스터_참조키',
  API_DTL_ID INT unsigned not null auto_increment COMMENT 'API_상세_기본키',
  COL_ID CHAR(30) NOT NULL                        COMMENT '항목ID',
  COL_NM VARCHAR(30) not null                     COMMENT '항목명',
  COL_TYPE VARCHAR(6) not null                    COMMENT '항목타입',
  REQ_YN CHAR(1) not null                         COMMENT '요청인자여부',
  PRIMARY KEY (API_DTL_ID),
  foreign KEY(API_MST_ID) references API_MST(API_MST_ID)
)
COMMENT='API_상세'
""")

# 3.1. API_TMP 삭제
dao.execute("drop table API_TMP")
# 3.2. API_TMP 생성
dao.execute("""
CREATE TABLE API_TMP (
  INF_ID CHAR(28) NOT NULL                        COMMENT '공공데이터ID',
  INF_NM VARCHAR(49) not null                     COMMENT '공공데이터명',
  CALL_URL VARCHAR(55) not null                   COMMENT '호출URL',
  COL_ID CHAR(30) NOT NULL                        COMMENT '항목ID',
  COL_NM VARCHAR(30) not null                     COMMENT '항목명',
  COL_TYPE VARCHAR(6) not null                    COMMENT '항목타입',
  REQ_YN CHAR(1) not null                         COMMENT '요청인자여부'
)
""")

In [ ]:
# DB 목록 출력하시오.
pd.DataFrame(dao.getDBList())

In [ ]:
# 테이블 목록을 출력하시오.
pd.DataFrame(dao.getTableList3('analysis'))

### [문제-06] API_TMP 테이블에 데이터 삭제 후 등록 하시오.

In [8]:
# 1단계: 데이터 삭제
sql = 'truncate table api_tmp'
print("삭제 후 건수: ", dao.execute(sql) )

# 2단계: 데이터 등록
sql = '''
insert into API_TMP(
  INF_ID, INF_NM, CALL_URL, COL_ID, COL_NM, COL_TYPE, REQ_YN
) VALUES(
  %s, %s, %s, %s, %s, %s, %s
)
'''
target_cnt = len(df) # 대상건수
proc_cnt = 0   # 처리건수
def save(param): # [등록 / 수정] 단건 처리
    return dao.execute(sql, param)

for v in df.values:
    proc_cnt += save(tuple(v))

# 3단계 : 처리 결과 출력
print("처리결과: ",{'대상건수':target_cnt, '처리건수':proc_cnt, '결과':(target_cnt==proc_cnt)})

0
{'대상건수': 25079, '처리건수': 25079, '결과': True}


### [문제-07] API_TMP 데이터를 분활해서 API_MST, API_DTL에 데이터를 등록 후 API_TMP 테이블을 제거하시오.

In [ ]:
# 1.1. api_mst 데이터 삭제
dao.execute("truncate table api_mst")
# 1.2. api_mst 등록
dao.execute("""
insert into api_mst(inf_id, inf_nm, call_url)
    select distinct inf_id, inf_nm, call_url -- distinct : 중복을 제거 한다.
    from api_tmp
)
""")

# 2.1. api_dtl 데이터 삭제
dao.execute("truncate table api_dtl")
# 2.2. api_dtl 등록
dao.execute("""
insert into api_dtl(api_mst_id, COL_ID, COL_NM, COL_TYPE, REQ_YN)
	select api_mst_id, COL_ID, COL_NM, COL_TYPE, REQ_YN
	  from api_tmp a
	 inner join api_mst b on a.inf_id = b.inf_id
)
""")

# 3. API_TMP 테이블 제거
dao.execute("drop table api_tmp")

### [데이터분석-01] 오픈 API의 총 갯수를 구하시오.

In [2]:
dao.row("select COUNT(*) as CNT from API_MST AM")

{'CNT': 1437}

### [데이터분석-02] 공공데이터명 별 항목 총 갯수를 구하시오. 항목갯수로 내림차순으로 정렬

In [6]:
pd.DataFrame(dao.rows("""
select A.INF_NM
     , COUNT(*) as CNT
  from API_MST A
 inner join API_DTL B on A.API_MST_ID = B.API_MST_ID  
 group by A.INF_NM
 order BY COUNT(*) DESC
""")).head(5)

,INF_NM,CNT
0,유통제품수거(먹는샘물) 수질검사결과,112
1,제조업체원수(먹는샘물) 수질검사결과,104
2,입찰공고목록 정보(공사부분),104
3,입찰공고목록 정보(용역부분),102
4,입찰공고목록 정보(물품부분),90


### [데이터분석-03] 데이터(항목) 타입별 총 갯수를 구하시오.
- number타입이 text타입의 1/3 이니 수치형 분석에 적합한 데이터는 작을것 같다.

In [8]:
pd.DataFrame(dao.rows("""
select COL_TYPE, COUNT(*) as CNT
  from API_DTL
 group by COL_TYPE
"""))

,COL_TYPE,CNT
0,NUMBER,6198
1,TEXT,18881


### [데이터분석-04] 공공데이터명을 항목타입이 텍스트보다 숫자형이 많은것을 기준으로 내림차순으로 정렬 하시오.
- 단, 텍스트 갯수가 더 많은 것을 빼시오.
- 수치형 항목이 많을 수록 수치분석에 적합할 확률이 높지 않을까?

In [10]:
pd.DataFrame(dao.rows("""
select INF_NM, TEXT_CNT, NUMBER_CNT, (NUMBER_CNT-TEXT_CNT) as NUMBER_DIFF
  from (
	select A.INF_NM
	     , SUM( case when COL_TYPE='TEXT' then 1 else 0 END ) as TEXT_CNT
	     , SUM( case when COL_TYPE='NUMBER' then 1 else 0 END ) as NUMBER_CNT
	  from API_MST A
	 inner join API_DTL B on A.API_MST_ID = B.API_MST_ID
	 group by A.INF_NM
   ) A1
  where (NUMBER_CNT-TEXT_CNT) >= 1 
  order by (NUMBER_CNT-TEXT_CNT) DESC
""")).head(5)

,INF_NM,TEXT_CNT,NUMBER_CNT,NUMBER_DIFF
0,특수학급 교원 집계현황,2,61,59
1,직위별 교원 현황,12,60,48
2,"등록장애인 집계현황(시군별, 유형별, 성별)",3,46,43
3,표시과목별 교원 현황(초),12,51,39
4,"졸업생의 진로 현황(중과정, 중학교)",13,51,38


### [데이터분석-05] 호출URL에서 API_KEY를 추출하여 출력하시오.SUBSTRING('문자열', 시작위치_숫자, 종료위치_숫자)

In [12]:
pd.DataFrame(dao.rows("""
select CALL_URL, SUBSTRING(CALL_URL2, INSTR(CALL_URL2,'/')+1, END_IDX) as AKP_KEY
  from (
	select CALL_URL, REPLACE(CALL_URL,'//','||') as CALL_URL2, LENGTH(CALL_URL) as END_IDX
	  from API_MST
) A1
""")).head(5)

,CALL_URL,AKP_KEY
0,https://openapi.gg.go.kr/Age0PrvuuseKidgartn,Age0PrvuuseKidgartn
1,https://openapi.gg.go.kr/PerpersonParkarea,PerpersonParkarea
2,https://openapi.gg.go.kr/AboveH30Mainbi,AboveH30Mainbi
3,https://openapi.gg.go.kr/PredictTreat,PredictTreat
4,https://openapi.gg.go.kr/CCTV,CCTV
